In [1]:
import cv2
import skvideo
import math
import os
import numpy as np
cv2.__version__

'3.4.1'

In [2]:
import imageio

In [3]:
import pickle
import matplotlib.pyplot as plt
%matplotlib inline


In [4]:
def CreateListofImages(folder, max_length=50, f_lists=np.array([]), period=1):
    """
    serve to make the list of video images with period 
    and max_length
    return list of images from folder of images
    """
    
    f_list = np.sort(os.listdir(folder))
    f_list = np.vectorize(lambda x: os.path.join(os.getcwd(), folder, x))(f_list)
    ind_mask = np.arange(len(f_list)) % period == 0
    
    
    while (np.sum(ind_mask) < max_length) & (period > 1):
        period -=1
        ind_mask = np.arange(len(f_list)) % period == 0
        if (period ==1) & (np.sum(ind_mask) < max_length):
            f_list = np.sort(np.random.choice(f_list, size=max_length))
            return np.append(f_lists, f_list)
        
    
    f_list = f_list[ind_mask]
    for i in range(len(f_list) - max_length):
        _f_list = f_list[i*max_length :(i+1)*max_length]
        if len(_f_list) == 0:
            break
        if len(_f_list) < max_length:
            _f_list = np.sort(np.random.choice(_f_list, size=max_length))
            break
        f_lists = np.append(f_lists, _f_list)
        
    return f_lists

In [30]:
from imageio import imread

def CreateTensorFromListofImages(image_list, max_length):
    """
    get the tensor from list of images
    """
    
    X = np.full((len(image_list), 197, 197, 3), 0, dtype=np.int32)
    for i,image in enumerate(image_list):
        X[i] = imread(image)
         
    return X

In [32]:
def Create_images_featues(list_videos, model, max_length=50):
    """
    from list of images build the data's features use model to get the image's features
    """
    im_features = np.array([])
    for f in list_videos:
        
        images_list = CreateListofImages(f, max_length=max_length)
        X = CreateTensorFromListofImages(images_list, max_length)
        X = X.reshape(X.shape[0]/max_length, max_length, 197, 197, 3)
        _im_features = np.full((X.shape[0], X.shape[1], 1, 1 ,2048), 0, dtype=np.int32)
        for i, x in enumerate(X):
            _im_features[i] = model.predict_on_batch(x.reshape(X.shape[1], X.shape[2], X.shape[3], X.shape[4]))
        if len(im_features) == 0:
            im_features = _im_features
        else:
            im_features = np.vstack((im_features, _im_features))
    return im_features

In [26]:
def Create_train_test(folder, test_size=0.1):
    """
    split the folder of video on train and test 
    """
    
    f_list = np.sort(os.listdir(os.path.join(os.getcwd(), folder)))
    f_list = np.vectorize(lambda x: os.path.join(os.getcwd(), folder, x))(f_list)
    _n_train = int(len(f_list) * test_size)
    _train_list = f_list[: len(f_list) -  _n_train]
    _test_list = f_list[len(f_list) -  _n_train :]
    
            
    return _train_list, _test_list

In [27]:
def get_train_test_data (folder_list, model, max_length=50, period=1, verbose=False):
    """
    serve to make the train test data as data and labels 
    folder_list - 2 folders contain video 2 labels
    model use to extract features from images
    max_length maximum length of images for one sapmle
    period - as take the image from video
    """
    for i,fd in enumerate(folder_list):
        if i ==0:
            train_video, test_video = Create_train_test(fd)
            if verbose:
                print 'test_size {}   {}'.format(len(test_video), fd)
            X_train = Create_images_featues(train_video, model)
            X_test = Create_images_featues(test_video, model)
            y_train = np.full(len(X_train), dtype=np.int8, fill_value=i)
            y_test = np.full(len(X_test), dtype=np.int8, fill_value=i)
        else:
            train_video, test_video = Create_train_test(fd)
            if verbose:
                print 'test_size {}  {}'.format(len(test_video), fd)
            _X_train = Create_images_featues(train_video, model)
            _X_test = Create_images_featues(test_video, model)
            X_train = np.concatenate((X_train, _X_train), axis=0)
            X_test = np.concatenate((X_test, _X_test), axis=0)
            y_train = np.concatenate((y_train, np.full(len(_X_train), dtype=np.int8, fill_value=i)), axis=0)
            y_test = np.concatenate((y_test, np.full(len(_X_test), dtype=np.int8, fill_value=i)), axis=0)
            
    return X_train, X_test, y_train, y_test

In [14]:
folders_video = ['video_S197/video_good', 'video_S197/video_bad']

In [10]:
#libraries
import tensorflow as tf
import tflearn
tf.__version__

/home/oleg/anaconda2/lib/python2.7/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


Instructions for updating:
Use the retry module or similar alternatives.


'1.7.0'

In [11]:
imRes = tf.keras.applications.ResNet50(include_top=False, input_shape=(197,197,3))

In [ ]:
imRes.predict_on_batch

In [33]:
X_train, X_test, y_train, y_test = get_train_test_data(folders_video, imRes, max_length=50, verbose=True)

test_size 5   video_S197/video_good
test_size 5  video_S197/video_bad


In [34]:
X_train.shape, y_train.shape, X_test.shape, y_test.shape

((689, 50, 1, 1, 2048), (689,), (60, 50, 1, 1, 2048), (60,))

In [35]:
X_train.nbytes/1024

275600

In [36]:
np.save(open('video_x_Res.npy', 'w'),X_train)
np.save(open('video_y_Res.npy','w'),y_train)
np.save(open('video_x_Res_val.npy', 'w'),X_test)
np.save(open('video_y_Res_val.npy','w'),y_test)

In [51]:
model = tf.keras.Sequential()
model.add(tf.keras.layers.Reshape((50,2048), input_shape=[50,1,1,2048] ))
model.add(tf.keras.layers.LSTM(50, dropout=0.4))
model.add(tf.keras.layers.Dense(1, activation='softmax'))

In [52]:
model.output_shape

(None, 1)

In [53]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
reshape_3 (Reshape)          (None, 50, 2048)          0         
_________________________________________________________________
lstm_3 (LSTM)                (None, 50)                419800    
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 51        
Total params: 419,851
Trainable params: 419,851
Non-trainable params: 0
_________________________________________________________________


In [54]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [55]:
with tf.device('/gpu:0'):
    model.fit(X_train, 
          y_train, 
          batch_size=8,  
          workers=8, 
                    epochs=50, 
                    verbose=2,
                    use_multiprocessing=True,
                    validation_data=(X_test ,   y_test)
        
                     )

Train on 689 samples, validate on 60 samples
Epoch 1/50
 - 6s - loss: 8.3530 - acc: 0.4761 - val_loss: 2.9228 - val_acc: 0.8167
Epoch 2/50
 - 5s - loss: 8.3530 - acc: 0.4761 - val_loss: 2.9228 - val_acc: 0.8167
Epoch 3/50
 - 5s - loss: 8.3530 - acc: 0.4761 - val_loss: 2.9228 - val_acc: 0.8167
Epoch 4/50
 - 5s - loss: 8.3530 - acc: 0.4761 - val_loss: 2.9228 - val_acc: 0.8167
Epoch 5/50
 - 5s - loss: 8.3530 - acc: 0.4761 - val_loss: 2.9228 - val_acc: 0.8167
Epoch 6/50
 - 5s - loss: 8.3530 - acc: 0.4761 - val_loss: 2.9228 - val_acc: 0.8167
Epoch 7/50
 - 5s - loss: 8.3530 - acc: 0.4761 - val_loss: 2.9228 - val_acc: 0.8167
Epoch 8/50
 - 5s - loss: 8.3530 - acc: 0.4761 - val_loss: 2.9228 - val_acc: 0.8167
Epoch 9/50
 - 5s - loss: 8.3530 - acc: 0.4761 - val_loss: 2.9228 - val_acc: 0.8167
Epoch 10/50
 - 5s - loss: 8.3530 - acc: 0.4761 - val_loss: 2.9228 - val_acc: 0.8167
Epoch 11/50
 - 5s - loss: 8.3530 - acc: 0.4761 - val_loss: 2.9228 - val_acc: 0.8167
Epoch 12/50
 - 5s - loss: 8.3530 - acc: 